In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql.types import *       # for datatype conversion
from pyspark.sql.functions import *   # for col() function
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import Tokenizer
import re
import pyspark.sql.functions as f
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from numpy import array
from math import sqrt
from pyspark.ml.clustering import KMeans
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import NGram
from pyspark.ml.feature import MaxAbsScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np
import matplotlib.pyplot as plt
from pyspark.ml.clustering import BisectingKMeans

sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Jeopardy Calculation") \
    .config("spark.executor.memory", '2g') \
    .config('spark.executor.cores', '1') \
    .config('spark.cores.max', '1') \
    .config("spark.driver.memory",'1g') \
    .getOrCreate()

from nltk.stem.porter import *
stemmer = PorterStemmer()
def stem(in_vec):
    out_vec = []
    for t in in_vec:
        t_stem = stemmer.stem(t)
        out_vec.append(t_stem)       
    return out_vec
from pyspark.sql.types import *
stemmer_udf = udf(lambda x: stem(x), ArrayType(StringType()))

In [2]:
#reading in the jeopardy data set
jeopardy = spark.read.json("JEOPARDY_QUESTIONS1-Copy1.json")
j_categoryCount = jeopardy.groupBy("category").count()
#This is the number of categories in the dataset which have greater than 100 observations
count100 = j_categoryCount.sort(desc("count")).filter(j_categoryCount["count"] > 100).count()
#This is a list of all categories which have a count greater than 100
top_categories = list(j_categoryCount.sort(desc("count")).select("category").limit(count100).toPandas().category)
#new dataset that only contains categories that have greater than 100 osbervations
jeo_f = jeopardy.where(col("category").isin(top_categories))

In [3]:
#stripping punctuation, tokenizing, stop word removing, and stemming for the modified dataset
jeo_fpunc = jeo_f.withColumn("stripped", f.regexp_replace(f.col("question"), "[\!@#$%^&*)(><,';:]", ""))
jeo_fpunc.cache()
tokenizer = Tokenizer(inputCol = "stripped", outputCol = "words")
tokenized_f = tokenizer.transform(jeo_fpunc)
tokenized_f.cache()
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
jeo_fStopRemoved = remover.transform(tokenized_f)
jeo_fStopRemoved.cache()
jeo_fStemmed = jeo_fStopRemoved.withColumn("stemmed", stemmer_udf("filtered"))
jeo_fStemmed.cache()

DataFrame[air_date: string, answer: string, category: string, question: string, round: string, show_number: string, value: string, stripped: string, words: array<string>, filtered: array<string>, stemmed: array<string>]

In [4]:
humanclust = [9, 0, 5, 7, 9, 7, 9, 2, 7, 8, 2, 2, 3, 1, 2, 8, 3, 2, 2, 5, 6, 5, 5, 8, 8, 5, 9, 4, 0, 6, 9, 5, 6, 5, 7, 1, 8, 0, 7, 9, 2, 8, 9, 8, 5, 6, 4, 6, 5, 9, 9, 5, 5, 8, 2, 6, 2, 7, 6, 4, 4, 0, 1, 9, 7, 0, 2, 1, 9, 7, 0, 4, 4, 4, 0, 0, 4, 2, 1, 0, 8, 9, 4, 8, 0, 1, 9, 9, 8, 8, 9, 0, 7, 0, 7, 4, 2, 9, 4, 4, 0, 7, 9, 0, 4, 4, 3, 2, 1, 9, 8, 4, 4, 9, 5, 9, 1, 0, 2, 4, 1, 9, 4, 4, 9, 6, 4, 4, 5, 4, 8, 9, 6, 5, 5, 0, 2, 0, 0, 4, 8, 8, 4, 7, 2]
human_udf = udf(lambda x: int(humanclust[top_categories.index(x)]))
jeo_g = jeo_fStemmed.withColumn("human", human_udf(jeo_f.category).cast("integer"))
jeo_g.show(5)

+----------+--------------------+--------------+--------------------+---------+-----------+-----+--------------------+--------------------+--------------------+--------------------+-----+
|  air_date|              answer|      category|            question|    round|show_number|value|            stripped|               words|            filtered|             stemmed|human|
+----------+--------------------+--------------+--------------------+---------+-----------+-----+--------------------+--------------------+--------------------+--------------------+-----+
|2004-12-31|          Copernicus|       HISTORY|'For the last 8 y...|Jeopardy!|       4680| $200|For the last 8 ye...|[for, the, last, ...|[last, 8, years, ...|[last, 8, year, l...|    7|
|2004-12-31|             the ant|3-LETTER WORDS|'In the title of ...|Jeopardy!|       4680| $200|In the title of a...|[in, the, title, ...|[title, aesop, fa...|[titl, aesop, fab...|    9|
|2004-12-31|      the Appian Way|       HISTORY|'Built in 31

In [5]:
comp_col = spark.read.csv("bkmf2.csv")
comp_col2 = comp_col.withColumn("ID", monotonically_increasing_id())
jeo_h = jeo_g.withColumn("ID", monotonically_increasing_id())
jeo_i = jeo_h.join(comp_col2, on="ID").withColumnRenamed("_c0", "computer")

In [6]:
jeo_i.select("computer").groupBy("computer").count().show(10)

+--------+-----+
|computer|count|
+--------+-----+
|       7| 1257|
|       3| 6297|
|       8|  224|
|       0| 3770|
|       5| 1844|
|       6| 3908|
|       9|  134|
|       1| 1566|
|       4| 3677|
|       2| 6575|
+--------+-----+



In [8]:
jeo_i.show(5)

+---+----------+--------------------+--------------+--------------------+---------+-----------+-----+--------------------+--------------------+--------------------+--------------------+-----+--------+
| ID|  air_date|              answer|      category|            question|    round|show_number|value|            stripped|               words|            filtered|             stemmed|human|computer|
+---+----------+--------------------+--------------+--------------------+---------+-----------+-----+--------------------+--------------------+--------------------+--------------------+-----+--------+
|  0|2004-12-31|          Copernicus|       HISTORY|'For the last 8 y...|Jeopardy!|       4680| $200|For the last 8 ye...|[for, the, last, ...|[last, 8, years, ...|[last, 8, year, l...|    7|       4|
|  1|2004-12-31|             the ant|3-LETTER WORDS|'In the title of ...|Jeopardy!|       4680| $200|In the title of a...|[in, the, title, ...|[title, aesop, fa...|[titl, aesop, fab...|    9|     

In [9]:
jeo_i.count()

29252

In [7]:
training,test = jeo_i.randomSplit([0.8,0.2], seed = 1) 

In [12]:
hashingTF = HashingTF(inputCol="stemmed", outputCol="rawFeatures", numFeatures = 50000)
idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features")
indexer = StringIndexer(inputCol="category", outputCol="label")
nb = NaiveBayes(modelType="multinomial", featuresCol = "features", labelCol = "label")

pipelineNB = Pipeline(stages=[hashingTF,idf,indexer,nb])
paramGrid = ParamGridBuilder() \
    .addGrid(nb.smoothing, [0.1,1.0,5.0,10.0,50.0]) \
    .build()

crossval = CrossValidator(estimator=pipelineNB,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=5)
cvModel = crossval.fit(training)

In [13]:
prediction = cvModel.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.28955427015991836


In [14]:
cvModel.avgMetrics

[0.2426881226816987,
 0.25906469856390973,
 0.26548632194893496,
 0.2645584721789181,
 0.25596038571836116]

In [15]:
prediction.groupBy('prediction').count().sort(desc("prediction")).show(145)

+----------+-----+
|prediction|count|
+----------+-----+
|     143.0|    4|
|     142.0|   13|
|     140.0|   11|
|     139.0|   10|
|     138.0|   16|
|     137.0|    4|
|     136.0|   10|
|     135.0|    2|
|     134.0|    8|
|     133.0|    7|
|     132.0|    6|
|     130.0|   14|
|     129.0|    9|
|     128.0|    8|
|     127.0|   13|
|     125.0|    3|
|     124.0|    1|
|     123.0|   12|
|     122.0|   10|
|     121.0|    5|
|     120.0|   10|
|     119.0|    4|
|     118.0|    1|
|     117.0|    5|
|     116.0|    1|
|     115.0|    5|
|     113.0|   13|
|     112.0|    5|
|     111.0|    9|
|     110.0|    8|
|     109.0|    3|
|     108.0|   19|
|     107.0|    3|
|     106.0|    4|
|     105.0|   12|
|     104.0|    7|
|     103.0|    7|
|     102.0|    3|
|     101.0|    3|
|     100.0|    8|
|      99.0|    3|
|      98.0|   20|
|      97.0|    9|
|      96.0|    2|
|      95.0|   17|
|      94.0|    4|
|      93.0|    7|
|      92.0|   10|
|      91.0|    3|
|      90.0|

In [16]:
prediction.filter(prediction['prediction'] == 1).select("category","question").sort("category") \
    .show(424,truncate=False)

+-----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|category               |question                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [17]:
hashingTF = HashingTF(inputCol="stemmed", outputCol="rawFeatures", numFeatures = 50000)
idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features")
indexer = StringIndexer(inputCol="human", outputCol="label")
nb = NaiveBayes(modelType="multinomial", featuresCol = "features", labelCol = "label")

pipelineNB = Pipeline(stages=[hashingTF,idf,indexer,nb])
paramGrid = ParamGridBuilder() \
    .addGrid(nb.smoothing, [0.1,1.0,5.0,10.0,50.0, 100]) \
    .build()

crossval = CrossValidator(estimator=pipelineNB,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=5)
cvModel = crossval.fit(training)

In [18]:
prediction = cvModel.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.5113984348417829


In [19]:
cvModel.avgMetrics

[0.4174288546786128,
 0.4604514426084799,
 0.49389347802839756,
 0.4995539022805334,
 0.5017680064876829,
 0.4952751057114002]

In [20]:
prediction.groupBy('prediction').count().sort(desc("prediction")).show(145)

+----------+-----+
|prediction|count|
+----------+-----+
|       9.0|    6|
|       8.0|  140|
|       7.0|  143|
|       6.0|  511|
|       5.0|  725|
|       4.0|  631|
|       3.0|  750|
|       2.0| 1019|
|       1.0|  709|
|       0.0| 1244|
+----------+-----+



In [21]:
prediction.filter(prediction['prediction'] == 0).select("category","question").sort("category") \
    .show(424,truncate=False)

+----------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|category              |question                                                                                                                                                                                                                                                                                                                                                                                           |
+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------

In [58]:
hashingTF = HashingTF(inputCol="stemmed", outputCol="rawFeatures", numFeatures = 50000)
idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features")
indexer = StringIndexer(inputCol="computer", outputCol="label")
nb = NaiveBayes(modelType="multinomial", featuresCol = "features", labelCol = "label")

pipelineNB = Pipeline(stages=[hashingTF,idf,indexer,nb])
paramGrid = ParamGridBuilder() \
    .addGrid(nb.smoothing, [5000, 20000, 30000, 40000, 50000]) \
    .build()

crossval = CrossValidator(estimator=pipelineNB,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=5)
cvModel = crossval.fit(training)

In [59]:
prediction = cvModel.transform(test)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(prediction)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.8123511398434842


In [60]:
cvModel.avgMetrics

[0.784590572212571,
 0.7861889733625345,
 0.7859240959371023,
 0.7853872439621085,
 0.7851015290621645]

In [67]:
prediction.groupBy('prediction').count().sort(desc("prediction")).show(145)

+----------+-----+
|prediction|count|
+----------+-----+
|       7.0|  183|
|       6.0|  179|
|       5.0|    3|
|       4.0|  595|
|       3.0|  882|
|       2.0| 1026|
|       1.0| 1597|
|       0.0| 1413|
+----------+-----+



In [68]:
prediction.filter(prediction['prediction'] == 0).select("category","question").sort("category") \
    .show(424,truncate=False)

+-----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|category               |question                                                                                                                                                                                                                                                                                                                                            |
+-----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------